In [6]:
import os
import shutil
import re
import itertools as it
os.chdir("/Users/BeckyMarcusMacbook/Thesis/EconTextCleaning/")
text_dir = "data/groups/E2/texts"
dest_dir = "data/temp/E2"
def reset():
    rm_ds(text_dir)
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    shutil.copytree(text_dir,dest_dir)
def rm_ds(dir):
    for file in os.listdir(dir):
        if file==".DS_Store":
            os.remove(os.path.join(dir,file))

In [ ]:
# jstor cover
# Every page except for the title and cover page had exactly one header line
# Page 00 was cover
# Page num at bottom in 1888-0-1, a couple other floating numbers we can get rid of
# only footnotes 1889 pages 1 and 24
# Title page:
    #1888: Get rid of top 5 lines
    #1889 get rid of top 2 lines, bottom 4
"""
Seeing always line 1 being removable:
    Sometimes also line 2, especially around 1900 when first or second line is 1-2 characters long
    We can also remove any line that is <=3 characters long, then the first line
Footnotes start with *
    1894 loves her
    Weird footnote on 1894-06:starts with '£500. T'
    1894-0-19: * doesn't show up, following line marked with t like character
        Maybe just look for line that ends with floating lowercase t and go after that
    Also some footnotes have their own line, they're not split, just extra long and wrapped
    Footnote in 1904, marked with '1 '
Some have page numbers on last line
    1894-0-18 had number on in line with the text, just at the end of it and separated by a lots of spaces
Capitalized author on last page of 1893, 1894, not 1891, but maybe can look for pattern
    {'1893':15, '1894':21}
1900 in rough shape, needs to be re-ocred
1901-01-03 has footnote too, marked with '1 '
    Honestly don't worry about that one
    
First pages:
1892: No lowercase in second line then good
1893,1894: ONly first line
1895: No lowercase until first real line
1900,1901, 1902: has <=2 lowercase letter in one of the title lines as an error
Works for all of them: Lines with <=2 lowercase letters on first page
"""


In [19]:
"""→for file in files:
→if page is ‘00’:
→remove page
"""
reset()
files = os.listdir(dest_dir)
files.sort()
for file in files:
    disc,year,num,pagetxt = file.split("-")
    page=int(pagetxt[:-4])
    if page==0:
        os.remove(os.path.join(dest_dir,file))

In [ ]:
"""→for page in pages:
get rid of jstor footer
→split text into lines
→strip whitespace from edges of each line
if not title page:
    Only Get everything after first line
    if length of last line is 1, delete that line
    if year = 1889 and page=24:
        get rid of the bottom line
else:
    if year is 1888:
        get rid of top 5 lines
    else:
        get rid of top 2, bottom 4 lines
join lines together
strip whole text
    
"""
files = os.listdir(dest_dir)
files.sort()
for file in files:
    if file[0]=='.':
        continue
    disc,year,num,pagetxt = file.split("-")
    page=int(pagetxt[:-4])
    path = os.path.join(dest_dir,file)
    
    text = open(path,'r').read()
    jstor_split = text.split("This content downloaded from")
    text =jstor_split[0].strip()
    lines = text.split("\n")
    lines = [l.strip() for l in lines]
    filtered_lines = [l for l in lines if len(l)>3]
    if page!=1:
        save_lines = filtered_lines[1:]
        if (year=='1893' and page==15 or year=='1894' and page==21):
            for i,line in enumerate(save_lines[::-1]):
                line_num = (i+1) *-1
                if re.search("[a-z]",line) is None:
                    save_lines = save_lines[:line_num]
                    break
    else:
        line_num=0
        lower_case_line=False
        any_upper =False
        while True:
            if len(re.findall(r'[a-z]',filtered_lines[line_num]))>2 and any_upper:
                break
            line_num+=1
            any_upper=True
        save_lines = filtered_lines[line_num:]

    text = "\n".join(save_lines)
    with open(path,'w') as f:
        f.write(text)

In [22]:
### Deal with line breaks across pages
files = os.listdir(dest_dir)
files.sort()
for first,second in it.pairwise(files):
    if first.split("-")[:3] != second.split("-")[:3]:
        continue
    path1 = os.path.join(dest_dir, first)
    path2 = os.path.join(dest_dir, second)

    with open(path1, 'r') as f1, open(path2, 'r') as f2:
        text1 = f1.read()
        text2 = f2.read()

    if text1.endswith("-"):
        first_word = re.match(r"^\S+", text2).group()
        new_text2 = re.sub(r"^\S+\s", "", text2)
        new_text1 = text1[:-1] + first_word

        with open(path1, 'w') as f1, open(path2, 'w') as f2:
            f1.write(new_text1.strip())
            f2.write(new_text2.strip())


In [23]:
### remove dash errors
def fix_dash_errors(text):
    new_text = re.sub(r"([a-zA-Z]+)\-+\n([a-zA-Z]+)([^\w\n\s])?", # Captures 3 groups: first half of word, second half of word, optional punctuation
                      r"\1\2\3\n", #removes dash and moves line break
                      text)
    new_text_lines_stripped=[line.strip() for line in new_text.split('\n')] #remove any extra leading or trailing whitespace
    return "\n".join(new_text_lines_stripped) #join lines back together
files = os.listdir(dest_dir)
files.sort()
for file in files:
    if file[0]=='.':
        continue
    path = os.path.join(dest_dir,file)
    text = open(path,'r').read()
    new_text = fix_dash_errors(text)
    open(path,'w').write(new_text)

### Handling footnotes

In [30]:
foot_lines = {}
step_temp = "manual_work/E4/.footnote_orig"
if os.path.exists(step_temp):
    shutil.rmtree(step_temp)
os.makedirs(step_temp)
files = os.listdir(dest_dir)
files.sort()
for file in files:
    fn_on_page = False
    path = os.path.join(dest_dir,file)
    save_path = os.path.join(step_temp,file)
    text = open(path,'r').read()
    lines = text.split("\n")
    if file == "Economics-1894-0-19.txt":
        for i,line in enumerate(lines):
            if re.search(r" t$",line):
                if not fn_on_page:
                    fn_on_page = True
                    foot_lines[file]=[]
                lines[i+1] = f"{i+1}-->{line}"
                foot_lines[file].append(i+1)
                break
    else:
        for i,line in enumerate(lines):
            if re.search(r"^(\*|£)",line) is not None:
                if not fn_on_page:
                    fn_on_page = True
                    foot_lines[file]=[]
                lines[i] = f"{i}-->{line}"
                foot_lines[file].append(i)
    if fn_on_page:
        new_text = "\n".join(lines)
        with open(save_path,'w') as f:
            f.write(new_text)

In [31]:
foot_lines

{'Economics-1893-0-01.txt': [28],
 'Economics-1894-0-05.txt': [29, 32],
 'Economics-1894-0-06.txt': [33],
 'Economics-1894-0-07.txt': [34],
 'Economics-1894-0-11.txt': [36],
 'Economics-1894-0-12.txt': [35],
 'Economics-1894-0-14.txt': [28],
 'Economics-1894-0-15.txt': [28],
 'Economics-1894-0-16.txt': [35],
 'Economics-1894-0-19.txt': [29],
 'Economics-1903-0-08.txt': [9]}

In [32]:
edited_foot_lines = {
    'Economics-1893-0-01.txt': [28],
 'Economics-1894-0-05.txt': [29, 32],
 'Economics-1894-0-06.txt': [33],
 'Economics-1894-0-07.txt': [34],
 'Economics-1894-0-11.txt': [36],
 'Economics-1894-0-12.txt': [35],
 'Economics-1894-0-14.txt': [28],
 'Economics-1894-0-15.txt': [28],
 'Economics-1894-0-16.txt': [35],
 'Economics-1894-0-19.txt': [29],
 'Economics-1903-0-08.txt': []}
for file,vals in edited_foot_lines.items():
    if vals!=[]:
        first_foot_line = min(vals)
        path = os.path.join(dest_dir,file)
        text = open(path,'r').read()
        lines = text.split("\n")
        lines_cut = lines[:first_foot_line]
        with open(path,'w') as f:
            f.write('\n'.join(lines_cut))